In [1]:
#Dependencies
import numpy as np
import os
import psycopg2
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
#Cloud DB Config
SQL_USERNAME = "postgres" #your personal username goes here
SQL_PASSWORD = "airlineproject" #your personal password goes here
DB_NAME = "airline_db" #The default name of the DB
DB_HOST = "airline-db.c0dqcvhs1y58.us-east-2.rds.amazonaws.com"
PORT = 5432
database_path = f"postgresql://{SQL_USERNAME}:{SQL_PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}"

In [3]:
from sqlalchemy import create_engine
engine = create_engine(database_path)

In [4]:
#AWS_DB_URL = (
    #os.environ.get('AWS_DB_URL')
    #.replace('postgres://', 'postgresql://', 1)
   # )
#engine = create_engine(AWS_DB_URL)
#connection = engine.connect()

In [5]:
#main_df = pd.read_sql_query('select Dep_Airport, Arr_Airport, Day_Week, month, Mkt_Ccode, Mkt_Fl_No, Arr_Delay_Time_Actual from sample' FETCH FIRST 10 ROWS ONLY, con=engine).head()

In [6]:
#main_df = pd.read_sql_query('select * from sample', con=engine).head()

In [7]:
main_df = pd.read_csv("sample.csv")
main_df.head()

,index,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,month,Arr_Delay_Time_Actual_Flag,Sched_Dep_Time_OAG_Block
0,48712,AA,1487.0,ORD,DEN,20180203,sat,2015,2023,2157,...,0,0,0,0,0,0,0,feb,0,evening
1,547480,WN,3389.0,LAS,BUR,20180107,sun,2055,2104,2155,...,0,0,0,0,0,0,0,jan,1,evening
2,592345,AA,4545.0,MIA,BNA,20180111,thu,1110,1108,1240,...,0,0,0,0,0,0,0,jan,0,late_morning
3,29934,AA,1911.0,DFW,PHL,20180220,tue,906,937,1315,...,0,0,0,0,0,0,0,feb,1,late_morning
4,615637,UA,3842.0,ORD,RST,20180130,tue,925,917,1042,...,0,0,0,0,0,0,0,jan,0,late_morning


In [8]:
main_df.drop(["index"], axis = 1, inplace = True)
main_df.head()

,Mkt_Ccode,Mkt_Fl_No,Dep_Airport,Arr_Airport,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,month,Arr_Delay_Time_Actual_Flag,Sched_Dep_Time_OAG_Block
0,AA,1487.0,ORD,DEN,20180203,sat,2015,2023,2157,2152,...,0,0,0,0,0,0,0,feb,0,evening
1,WN,3389.0,LAS,BUR,20180107,sun,2055,2104,2155,2208,...,0,0,0,0,0,0,0,jan,1,evening
2,AA,4545.0,MIA,BNA,20180111,thu,1110,1108,1240,1225,...,0,0,0,0,0,0,0,jan,0,late_morning
3,AA,1911.0,DFW,PHL,20180220,tue,906,937,1315,1331,...,0,0,0,0,0,0,0,feb,1,late_morning
4,UA,3842.0,ORD,RST,20180130,tue,925,917,1042,1024,...,0,0,0,0,0,0,0,jan,0,late_morning


In [9]:
#Importing lat lng and location files
airports = pd.read_csv("airports.csv")
airport_data = pd.read_csv("airport-codes.csv")
clean_airport_data = airport_data.dropna(how="all")
#clean_airport_data2 = clean_airport_data.drop(labels="Unnamed: 7", axis=1)

In [10]:
main_df2 = main_df.merge(airports, how="left", on="Dep_Airport")
main_arr_df2 = main_df.merge(airports, how="left", on="Arr_Airport")

In [11]:
main_df3 = main_df2.merge(clean_airport_data, how="left", on="Dep_Airport")
main_arr_df3 = main_arr_df2.merge(clean_airport_data, how="left", on="Arr_Airport")

In [12]:
main_arr_df3.head()
main_arr_df3.rename(columns = {"City name_y": "arr_city_name"}, inplace = True)

In [13]:
main_arr_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 44 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Mkt_Ccode                               200000 non-null  object 
 1   Mkt_Fl_No                               200000 non-null  float64
 2   Dep_Airport_x                           200000 non-null  object 
 3   Arr_Airport                             200000 non-null  object 
 4   Date                                    200000 non-null  int64  
 5   Day_Week                                200000 non-null  object 
 6   Sched_Dep_Time_OAG                      200000 non-null  int64  
 7   Gate_Dep_Time_Actual                    200000 non-null  int64  
 8   Sched_Arr_Time_OAG                      200000 non-null  int64  
 9   Gate_Arr_Time_Actual                    200000 non-null  int64  
 10  Sched_Elapsed_Time_CRS                  2000

In [14]:
means = main_df3.groupby('Dep_Airport').mean()
means_arr = main_arr_df3.groupby('Arr_Airport').mean()

In [15]:
means.head()

,Mkt_Fl_No,Date,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,Sched_Elapsed_Time_CRS,Actual_Gate_Gate_Time,Dep_Delay_Time_Actual,Arr_Delay_Time_Actual,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,Arr_Delay_Time_Actual_Flag,lat,lng
Dep_Airport,,,,,,,,,,,,,,,,,,,,,
ABE,4214.654321,2.018014e+07,1188.024691,1220.975309,1307.888889,1323.271605,64.061728,62.296296,25.666667,24.975309,...,2.827160,0.0,11.432099,0.000000,0.000000,0.000000,0.0,0.333333,40.652100,-75.440804
ABQ,3811.127566,2.018014e+07,1206.763930,1170.435484,1412.247801,1368.234604,113.489736,104.719941,3.582111,-1.442815,...,2.979472,0.0,3.788856,0.041056,0.002933,0.002933,0.0,0.259531,35.040199,-106.609001
ABY,3316.384615,2.018021e+07,1081.346154,1117.743590,1184.846154,1226.923077,60.935897,63.538462,31.717949,35.102564,...,3.961538,0.0,25.794872,0.000000,0.000000,0.000000,0.0,0.602564,31.535500,-84.194504
ADK,161.000000,2.018022e+07,1715.000000,1482.142857,2057.000000,1813.714286,162.000000,143.000000,12.142857,16.285714,...,3.571429,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.857143,51.877998,-176.645996
ADQ,43.000000,2.018022e+07,1363.750000,1280.857143,1413.660714,1368.214286,49.910714,50.214286,-7.500000,-5.428571,...,0.553571,0.0,6.553571,0.000000,0.000000,0.000000,0.0,0.178571,57.750000,-152.494003


In [16]:
means_arr.head()

,Mkt_Fl_No,Date,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,Gate_Arr_Time_Actual,Sched_Elapsed_Time_CRS,Actual_Gate_Gate_Time,Dep_Delay_Time_Actual,Arr_Delay_Time_Actual,...,Mins_Late_Delay_Code_G_NAS,Mins_Late_Delay_Code_H_Security,Mins_Late_Delay_Code_I_Late_Arr_Flight,First_Gate_Dep_Time,Total_Ground_Away_From_Gate,Longest_Ground_Time_Away_From_Gate,No_Landings_At_Div_Airport,Arr_Delay_Time_Actual_Flag,lat,lng
Arr_Airport,,,,,,,,,,,,,,,,,,,,,
ABE,4162.538462,2.018013e+07,1727.115385,1701.051282,1815.320513,1764.948718,53.846154,46.538462,4.115385,-2.897436,...,0.307692,0.0,2.410256,21.102564,0.961538,0.961538,0.000000,0.307692,40.652100,-75.440804
ABQ,4161.613032,2.018014e+07,1423.691489,1377.827128,1543.634309,1463.244681,112.948138,106.191489,6.603723,3.352394,...,1.488032,0.0,3.315160,4.864362,0.409574,0.409574,0.011968,0.373670,35.040199,-106.609001
ABY,3501.850746,2.018021e+07,1537.746269,1548.791045,1631.358209,1621.029851,56.000000,84.179104,9.253731,37.432836,...,0.776119,0.0,2.820896,0.000000,0.000000,0.000000,0.044776,0.417910,31.535500,-84.194504
ADK,164.000000,2.018022e+07,1412.500000,1422.500000,1618.333333,1626.666667,185.833333,184.166667,3.333333,1.666667,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,51.877998,-176.645996
ADQ,48.061224,2.018021e+07,1088.918367,980.591837,1187.775510,1072.530612,58.857143,59.285714,-2.734694,3.979592,...,5.693878,0.0,1.836735,0.000000,0.000000,0.000000,0.000000,0.346939,57.750000,-152.494003


In [17]:
mean_merge = means.filter(['Dep_Airport', "Arr_Delay_Time_Actual"], axis=1)
mean_merge.rename(columns={"Arr_Delay_Time_Actual": "mean_arr_delay"}, inplace = True)
mean_merge.head()

,mean_arr_delay
Dep_Airport,
ABE,24.975309
ABQ,-1.442815
ABY,35.102564
ADK,16.285714
ADQ,-5.428571


In [18]:
mean_arr_merge = means_arr.filter(['Arr_Airport', "Arr_Delay_Time_Actual"], axis=1)
mean_arr_merge.rename(columns={"Arr_Delay_Time_Actual": "mean_arr_delay"}, inplace = True)
mean_arr_merge.head()

,mean_arr_delay
Arr_Airport,
ABE,-2.897436
ABQ,3.352394
ABY,37.432836
ADK,1.666667
ADQ,3.979592


In [19]:
final = mean_merge.merge(main_df3, how="left", on="Dep_Airport")
final2 = final.drop_duplicates(subset="Dep_Airport")
final2.head()

,Dep_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Arr_Airport_x,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Arr_Airport_y,lat,lng,City name_y,Airport Code,Airport Name_y,Country name_y,Country Abbrev.,World Area Code,Arr_Airport
0,ABE,24.975309,AA,4177.0,PHL,20180119,fri,620,610,707,...,ABE,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67,ABE
81,ABQ,-1.442815,WN,462.0,SAN,20180131,wed,1610,1712,1705,...,ABQ,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67,ABQ
763,ABY,35.102564,DL,3280.0,ATL,20180213,tue,1759,1750,1907,...,ABY,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67,ABY
841,ADK,16.285714,AS,161.0,ANC,20180224,sat,1715,1731,2057,...,ADK,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67,ADK
848,ADQ,-5.428571,AS,40.0,ANC,20180206,tue,1702,1640,1752,...,ADQ,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67,ADQ


In [20]:
final_arr = mean_arr_merge.merge(main_arr_df3, how="left", on="Arr_Airport")
final2_arr = final_arr.drop_duplicates(subset="Arr_Airport")
final2_arr.head()

,Arr_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Dep_Airport_x,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Dep_Airport_y,lat,lng,arr_city_name,Airport Code,Airport Name_y,Country name_y,Country Abbrev.,World Area Code,Dep_Airport
0,ABE,-2.897436,AA,4175.0,PHL,20180123,tue,1540,1540,1624,...,ABE,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67,ABE
78,ABQ,3.352394,WN,2674.0,PHX,20180121,sun,1255,1254,1400,...,ABQ,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67,ABQ
830,ABY,37.432836,DL,5004.0,ATL,20180218,sun,2107,2057,2200,...,ABY,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67,ABY
897,ADK,1.666667,AS,164.0,ANC,20180228,wed,1413,1401,1617,...,ADK,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67,ADK
903,ADQ,3.979592,AS,49.0,ANC,20180218,sun,1515,1503,1613,...,ADQ,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67,ADQ


In [21]:
final2.rename(columns = {"City name_x" : "city_name", "Country name_y": "country_name"}, inplace = True)
final2.dropna(subset=['country_name'])

C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Dep_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Arr_Airport_x,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Arr_Airport_y,lat,lng,City name_y,Airport Code,Airport Name_y,country_name,Country Abbrev.,World Area Code,Arr_Airport
0,ABE,24.975309,AA,4177.0,PHL,20180119,fri,620,610,707,...,ABE,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67,ABE
81,ABQ,-1.442815,WN,462.0,SAN,20180131,wed,1610,1712,1705,...,ABQ,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67,ABQ
763,ABY,35.102564,DL,3280.0,ATL,20180213,tue,1759,1750,1907,...,ABY,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67,ABY
841,ADK,16.285714,AS,161.0,ANC,20180224,sat,1715,1731,2057,...,ADK,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67,ADK
848,ADQ,-5.428571,AS,40.0,ANC,20180206,tue,1702,1640,1752,...,ADQ,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67,ADQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199514,VLD,0.343750,DL,3383.0,ATL,20180222,thu,1143,1149,1303,...,VLD,30.782499,-83.276703,Valdosta,VLD,Regional,United States,US,67,VLD
199578,VPS,0.179245,AA,2283.0,DFW,20180214,wed,1326,1321,1543,...,VPS,30.483200,-86.525398,Valparaiso,VPS,Ft. Walton Beach,United States,US,67,VPS
199684,WRG,-0.807692,AS,64.0,KTN,20180206,tue,1623,1703,1654,...,WRG,56.484299,-132.369995,Wrangell,WRG,Wrangell SPB,United States,US,67,WRG
199736,XNA,32.809045,AA,4388.0,LGA,20180129,mon,1734,1758,2129,...,XNA,36.281898,-94.306801,Fayetteville,XNA,Northwest Arkansas Rgn,United States,US,67,XNA


In [22]:
final2_arr.dropna(subset=['arr_city_name'], inplace = True)
final2_arr.head()

C:\Users\dlken\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Arr_Airport,mean_arr_delay,Mkt_Ccode,Mkt_Fl_No,Dep_Airport_x,Date,Day_Week,Sched_Dep_Time_OAG,Gate_Dep_Time_Actual,Sched_Arr_Time_OAG,...,Dep_Airport_y,lat,lng,arr_city_name,Airport Code,Airport Name_y,Country name_y,Country Abbrev.,World Area Code,Dep_Airport
0,ABE,-2.897436,AA,4175.0,PHL,20180123,tue,1540,1540,1624,...,ABE,40.652100,-75.440804,Allentown,ABE,Lehigh Valley International,United States,US,67,ABE
78,ABQ,3.352394,WN,2674.0,PHX,20180121,sun,1255,1254,1400,...,ABQ,35.040199,-106.609001,Albuquerque,ABQ,Albuquerque International,United States,US,67,ABQ
830,ABY,37.432836,DL,5004.0,ATL,20180218,sun,2107,2057,2200,...,ABY,31.535500,-84.194504,Albany,ABY,Dougherty County,United States,US,67,ABY
897,ADK,1.666667,AS,164.0,ANC,20180228,wed,1413,1401,1617,...,ADK,51.877998,-176.645996,Adak Island,ADK,Adak Island NS,United States,US,67,ADK
903,ADQ,3.979592,AS,49.0,ANC,20180218,sun,1515,1503,1613,...,ADQ,57.750000,-152.494003,Kodiak,ADQ,Kodiak Airport,United States,US,67,ADQ


In [23]:
column = final2_arr["mean_arr_delay"]
max_value = column.max()
print(max_value)
min_value = column.min()
print(min_value)

54.09803921568628
-18.895027624309392


In [24]:
#Function for Creating geojson from df
def df_to_geojson(df, properties, lat='lat', lon='lng'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [25]:
cols = ['Dep_Airport', "city_name", "mean_arr_delay"]
geojson = df_to_geojson(final2, cols)

In [26]:
output_filename = 'airport.geojson'
with open(output_filename, 'w') as json_file:
    json.dump(geojson, json_file)

In [27]:
final2.to_csv('final2.csv')

In [28]:
final2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 224 entries, 0 to 199935
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Dep_Airport                             224 non-null    object 
 1   mean_arr_delay                          224 non-null    float64
 2   Mkt_Ccode                               224 non-null    object 
 3   Mkt_Fl_No                               224 non-null    float64
 4   Arr_Airport_x                           224 non-null    object 
 5   Date                                    224 non-null    int64  
 6   Day_Week                                224 non-null    object 
 7   Sched_Dep_Time_OAG                      224 non-null    int64  
 8   Gate_Dep_Time_Actual                    224 non-null    int64  
 9   Sched_Arr_Time_OAG                      224 non-null    int64  
 10  Gate_Arr_Time_Actual                    224 non-null    int

In [29]:
#Creating geojson based upon arrival airport groupby
cols = ['Arr_Airport', "arr_city_name", "mean_arr_delay"]
geojson = df_to_geojson(final2_arr, cols)
output_filename = 'arr_airport.geojson'
with open(output_filename, 'w') as json_file:
    json.dump(geojson, json_file)